In [1]:
from astropy.io import fits
import numpy as np
from astropy.table import QTable,vstack
import astropy.units as u
import astropy.utils
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import matplotlib.colors
from scipy.stats import chisquare
from scipy.stats import chi2
from scipy.stats import lognorm
from scipy.stats import f
from astropy.timeseries import LombScargle
import time
from astropy.visualization import hist
from scipy.optimize import curve_fit
from astropy.timeseries import BoxLeastSquares
from astropy.timeseries import TimeSeries
from astropy.time import Time
from tqdm import tqdm


import functions
#import upsilon

In [2]:
gaia_zr=QTable.read('ztf_gaia_zr.csv')
gaia_zg=QTable.read('ztf_gaia_zg.csv')

In [19]:
t_zr=QTable.read('t_zr_0222.ecsv')
t_zg=QTable.read('t_zg_0222.ecsv')

In [20]:
t_zr.add_column(np.zeros(len(t_zr)), name='GaiaDist', index=-8)
t_zr.add_column(np.zeros(len(t_zr),dtype=int), name='multiflag', index=-9)
t_zr.add_column(np.zeros(len(t_zr),dtype=int), name='Plxflag', index=-4)
t_zr.add_column(np.zeros(len(t_zr)), name='fluxratio', index=-9)

t_zg.add_column(np.zeros(len(t_zg)), name='GaiaDist', index=-8)
t_zg.add_column(np.zeros(len(t_zg),dtype=int), name='multiflag', index=-9)
t_zg.add_column(np.zeros(len(t_zg),dtype=int), name='Plxflag', index=-4)
t_zg.add_column(np.zeros(len(t_zg)), name='fluxratio', index=-9)

In [21]:
t_zr

SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type,Gaia_count,multiflag,GaiaDist,fluxratio,Gmag,RFG,Plx,e_Plx,Plxflag,var,lc_var,var_flag
int32,str2,str16,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,str14,float64,float64,float64,str37,float64,str15,int32,int32,float64,float64,float64,float64,float64,float64,int32,str1,str1,bool
1,zr,806203100023452,5.0720632,59.3038752,851,427.1630554199219,1.4524961212923433,0.9999958153308316,0.015309100970625877,3.1752911585385783e-17,2800.0,0.997336033066433,0.19455600160117095,0.828718121558811,15.05040168762207,0.015309100970625877,14.99232292175293,15.130514144897461,-0.0011725469958037138,1.2038142681121826,0.48235724946974495,0.14902942755047316,2.824592533146398,1.0,0.05488081763402275,0.0016871191344681735,0.9836370283290529,g1,NonVar,0.6,0.0,1511.901550900002,[CHM2007] LDC 31 J002017.33+5918136,0.734832,GinGroup,1,0,0.0,0.0,15.114388,1472.66,0.3951,0.1062,0,N,N,False
1,zr,1809211300007650,5.0720654,59.3038596,77,299.4801025390625,1.1851211036605616,0.9981997509944572,0.011568024754524231,0.11454843350744998,0.10272337615237741,2800.0,0.21220607908278846,0.014182862102960344,15.044246673583984,0.011568024754524231,15.019607543945312,15.088167190551758,-0.0011076524388045073,1.0815422534942627,7.691495349172446,11.43936826330771,16.4795450267754,1.0,0.8676756539226124,0.2525176926349306,0.8319298278786083,g1,DSCT,0.27,1.0,0.10272079145728628,[CHM2007] LDC 31 J002017.33+5918136,0.734832,GinGroup,1,0,0.0,0.0,15.114388,1472.66,0.3951,0.1062,0,N,N,False
2,zr,1809211300070217,5.0690426,59.3040475,56,20.568172454833984,1.4475981996385179,0.9386170974691139,0.15365347266197205,0.1074781086890979,0.37890453869539864,2800.0,0.2778420325947693,0.037633445760631774,20.251352310180664,0.15365347266197205,19.95233154296875,20.715749740600586,-0.12701313197612762,1.2117805480957031,0.12220204058816697,0.15784787133740305,0.09753925664793019,1.0,0.7175947795787108,0.9507177794597677,0.3868711012969295,gu,NonVar,0.44,1.0,0.03860415050793573,--,nan,--,1,0,0.0,0.0,20.248793,199.839,-0.7083,0.6045,0,N,N,False
2,zr,806203100065936,5.0690334,59.3040661,501,32.86368179321289,1.720545692801813,0.9582426406442783,0.15914002060890198,0.014313439395987624,0.29343072946306986,0.997343815080248,0.08066567509773769,0.003899007689213132,20.25907325744629,0.15914002060890198,19.6058349609375,20.76239013671875,-0.1274305284023285,1.329569697380066,0.008008459690955328,0.00843750045799339,0.039886978110141094,1.0,0.5222407916467862,0.02626604237364416,0.9748122646586905,g1,NonVar,0.69,1.0,0.03133595528004489,--,nan,--,1,0,0.0,0.0,20.248793,199.839,-0.7083,0.6045,0,N,N,False
3,zr,806203100057022,5.0755433,59.3020667,716,37.69181442260742,1.571952371493106,0.9992624947012461,0.11341223865747452,0.06381223460712625,0.11947880733901506,0.997336033066433,0.05450309054787811,0.006059245845272846,19.72352409362793,0.11341223865747452,19.399478912353516,20.101001739501953,-0.0998307392001152,1.263882040977478,0.04066865471004858,0.01812575768444976,0.1534866931895394,1.0,0.11074222557864655,0.008343830861197277,0.9495883798098617,g1,NonVar,0.57,1.0,0.2284414330244274,--,nan,--,1,0,0.0,0.0,19.72043,317.362,0.4084,0.3136,0,N,N,False
4,zr,806203100065940,5.0739557,59.3015362,37,16.96639633178711,2.6638452457437145,0.12783449129189933,0.2826226055622101,0.000256625330625918,0.18071546460548857,0.9972560997935076,0.727522610000513,0.4713412770901734,21.201833724975586,0.2826226055622101,20.41510009765625,21.721399307250977,-0.16652198135852814,1.6300477981567383,0.15947146803044537,0.12934442394467066,0.18433703530765855,0.0,0.33379363946

In [24]:
for i,row in enumerate(t_zr):
    common=gaia_zr[gaia_zr['SourceID']==row['SourceID']]
    row['Gaia_count']=len(common)
    if len(common)!=0:
        match=common[common['Gmag'].argmin()]
        row['GaiaDist']=match['angDist']
        row['Gmag']=match['Gmag']
        row['RFG']=match['RFG']
        row['Plx']=match['Plx']
        row['e_Plx']=match['e_Plx']
        if (match['Plx']>0) and (match['Plx']>match['e_Plx']):
            row['Plxflag']=1
    if len(common)>=2:
        magsort=np.array(common['Gmag'])
        magsort.sort()
        # 2.5^2.5=10  2.5^5=100
        row['fluxratio']=2.5**(magsort[1]-magsort[0])
        if (magsort[1]-magsort[0])<2.5:
            row['multiflag']=1
        

for i,row in enumerate(t_zg):
    common=gaia_zg[gaia_zg['SourceID']==row['SourceID']]
    row['Gaia_count']=len(common)
    if len(common)!=0:
        match=common[common['Gmag'].argmin()]
        row['GaiaDist']=match['angDist']
        row['Gmag']=match['Gmag']
        row['RFG']=match['RFG']
        row['Plx']=match['Plx']
        row['e_Plx']=match['e_Plx']
        if (match['Plx']>0) and (match['Plx']>match['e_Plx']):
            row['Plxflag']=1
    if len(common)>=2:
        magsort=np.array(common['Gmag'])
        magsort.sort()
        row['fluxratio']=2.5**(magsort[1]-magsort[0])
        if (magsort[1]-magsort[0])<2.5:
            row['multiflag']=1

D:\Anaconda3\lib\site-packages\astropy\table\column.py:1169: UserWarning: Warning: converting a masked element to nan.
  self.data[index] = value


In [25]:
t_zr[t_zr['multiflag']==True]

SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type,Gaia_count,multiflag,GaiaDist,fluxratio,Gmag,RFG,Plx,e_Plx,Plxflag,var,lc_var,var_flag
int32,str2,str16,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,str14,float64,float64,float64,str37,float64,str15,int32,int32,float64,float64,float64,float64,float64,float64,int32,str1,str1,bool
49,zr,1809211300054363,5.0608969,59.3078032,37,16.34441375732422,1.727326132042372,0.7010528649758558,0.20339244604110718,0.1674990113566612,33.194528343939744,1.000079030206485,0.36127578314867737,0.14249080734625313,20.6938419342041,0.20339244604110718,20.227693557739258,21.068403244018555,-0.12887901067733765,1.3165677785873413,0.5811592944441982,0.8194620732716169,0.5502238728651196,0.0,0.810900598898151,0.8691202891040376,0.472165984921352,gu,DSCT,0.53,1.0,0.03027512764973706,--,nan,--,2,1,0.243022,1.4012702739397054,20.777805,94.3147,0.5504,1.333,0,N,N,False
55,zr,806203100043559,5.0723453,59.3111226,855,1322.654541015625,28.139266597643587,0.0,0.09289497882127762,1.0401704349953329e-06,2800.0,0.997336033066433,0.09657835576966833,0.8283801394042668,16.962669372558594,0.09289497882127762,16.404556274414062,17.23210906982422,-0.1622304767370224,5.306269645690918,0.009358681227241613,0.001534619994822164,0.12127946984456363,1.0,0.015938986578472036,9.779189688419176e-05,0.9984113822662893,g1,NonVar,0.79,1.0,2267.852326350003,[BGR2000] Br 1,0.17367,HII,2,1,0.884379,1.1336349112350974,20.799721,47.8212,nan,nan,0,V,V,False
55,zr,1809211300066648,5.0724769,59.3111324,77,649.3062133789062,17.884333922325375,9.82310740598561e-98,0.06559567898511887,0.00021590831277816652,0.10674401734567869,2800.0,0.34204672407082565,0.021634072466851725,16.935136795043945,0.06559567898511887,16.740747451782227,17.062875747680664,-0.09860587120056152,4.204975605010986,2.648841206633338,0.4319534830074969,2.0249945408637027,1.0,0.015766331876517103,0.026698985916089324,0.3660444451778835,gu,DSCT,0.45,1.0,0.0964498720601044,[BGR2000] Br 1,0.17367,HII,2,1,0.884379,1.1336349112350974,20.799721,47.8212,nan,nan,0,V,V,False
58,zr,806203100065929,5.0600291,59.3078404,174,30.258071899414062,4.208894386162636,4.6261534917462915e-11,0.29820770025253296,0.0009325500345800615,377.81088885717867,0.9973072933112778,0.2197682422653083,0.6556185882886443,20.766319274902344,0.29820770025253296,19.751018524169922,21.671676635742188,-0.2694375216960907,2.109649181365967,0.03802916936076219,0.007997569512278382,0.05990628372462142,1.0,0.025967093123118623,0.010630232273520779,0.7182855397362811,g1,NonVar,0.7,1.0,0.031467081274149024,--,nan,--,2,1,1.822076,1.4012702739397054,20.777805,94.3147,0.5504,1.333,0,V,V,False
61,zr,806203100057086,5.0673962,59.2966648,763,119.71590423583984,2.620045275938775,2.6526046263180732e-05,0.08889435231685638,0.006552464365108978,0.1586808435484777,0.997336033066433,0.06326911738485413,0.013481477002830613,19.266685485839844,0.08889435231685638,18.593414306640625,19.563026428222656,-0.09122668951749802,1.6262627840042114,0.026833046086836145,0.0006577322226745911,0.12968060739306825,1.0,0.0003661386938569322,1.5734514393295207e-05,0.9727637763562661,g1,NonVar,0.59,1.0,0.030438315131565745,--,nan,--,2,1,0.323213,1.6046459566062432,19.862377,194.461,1.8337,0.6264,1,N,N,False
61,zr,1809211300054524,5.0674362,59.2966473,73,50.554378509521484,1.741231209832435,0.7752807341412313,0.07800210267305374,0.14625082623450397,0.1734569855308386,2800.0,0.21557982419575988,0.11693446224323409,19.273544311523438,0.07800210267305374,19.050844192504883,19.426868438720703,-0.07320504635572433,1.3168590068817139,0.5305727037796134,0.8392106

In [26]:
t_zg[t_zg['multiflag']==True]

SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type,Gaia_count,multiflag,GaiaDist,fluxratio,Gmag,RFG,Plx,e_Plx,Plxflag,var,lc_var,var_flag
int32,str2,str16,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,str14,float64,float64,float64,str37,float64,str15,int32,int32,float64,float64,float64,float64,float64,float64,int32,str1,str1,bool
25,zg,806103100042687,5.0693889,59.3087403,237,92.99468231201172,7.836505926956435,1.512161804629138e-83,0.2091728299856186,1.3050029596712171e-13,365.20933973392897,1.0000143393856216,0.27259997355979004,0.4319465018424059,20.036285400390625,0.2091728299856186,18.759004592895508,20.42414093017578,-0.3137721121311188,2.870281219482422,0.005032831736797012,0.0007655728812316515,0.030898521657569875,1.0,0.013749444378141296,0.00037407861582017254,0.9856220212643277,g1,CEPH_Other,0.65,1.0,0.9947116227631579,--,nan,--,2,1,1.973932,1.1650830777292955,20.466309,132.148,1.6057,0.7252,1,V,V,False
25,zg,1809111300047902,5.0694621,59.3088523,36,40.49337387084961,5.178225498591696,8.048144074547564e-07,0.22259026765823364,0.00047935979507551194,0.17544479025567672,2800.0,0.5728030200438375,0.04952022327252487,20.041297912597656,0.22259026765823364,19.389726638793945,20.490066528320312,-0.3091849982738495,2.317760705947876,0.06488461442003488,0.021163611004506994,0.059744777164266294,1.0,0.060936121212942895,0.0714271704005417,0.45805178479963216,gu,DSCT,0.73,1.0,0.11622198129980171,--,nan,--,2,1,1.973932,1.1650830777292955,20.466309,132.148,1.6057,0.7252,1,V,V,False
55,zg,806103100036162,5.0724243,59.3110593,337,524.6041259765625,11.53433716011322,3.585358205401398e-226,0.09109293669462204,6.624513205261614e-05,0.9972441069114794,1.0000143393856216,0.09943739652561774,0.904815666787983,17.759519577026367,0.09109293669462204,17.35228729248047,18.282495498657227,-0.11940788477659225,3.39772629737854,0.021082675533131933,0.0009674376856350767,0.058600720395744474,1.0,0.0012778740354683071,0.0001663393556028403,0.8995923437308072,g1,RRL_ab,0.53,1.0,0.4986132790810157,[BGR2000] Br 1,0.179192,HII,2,1,0.977562,1.1336349112350974,20.799721,47.8212,nan,nan,0,V,V,False
55,zg,1809111300028565,5.07245,59.3110868,66,392.635986328125,14.967355206033764,8.396331195386733e-66,0.09816054254770279,0.001972449838350469,0.22232818693454928,2800.0,0.33767960153347515,0.09093510017475841,17.740047454833984,0.09816054254770279,17.36347198486328,17.94055938720703,-0.16121967136859894,3.8505306243896484,0.379122196310376,0.1487498025445291,0.17974030374835914,1.0,0.0868797174594248,0.34586527658886057,0.1729739946754486,gu,DSCT,0.5,1.0,0.05968903932818899,[BGR2000] Br 1,0.179192,HII,2,1,0.977562,1.1336349112350974,20.799721,47.8212,nan,nan,0,V,V,False
61,zg,806103100040276,5.067418,59.2966828,127,34.537837982177734,2.9826919894988544,0.0002645315332656795,0.2247643619775772,3.7800404304745114e-06,1.0321732994604866,1.0000110121384305,0.269655655474641,0.17275268370836522,20.80208396911621,0.2247643619775772,19.886661529541016,21.27700424194336,-0.16201622784137726,1.7484934329986572,0.06429252948295217,0.027821371972721838,0.09577891675859604,0.0,0.10470883703559108,0.04870495303435052,0.694010862155763,g1,CEPH_Other,0.55,1.0,1.0322700696219942,--,nan,--,2,1,0.380721,1.6046459566062432,19.862377,194.461,1.8337,0.6264,1,N,N,False
61,zg,1809111300042664,5.067473,59.2966488,15,13.316123962402344,1.3295706010789117,0.8108960576869506,0.18107299506664276,0.04263363261093376,0.10109222043491486,2800.0,0.7602083559250822,0.044869790029799385,20.748355865478516,0.18107299506664276,20.361892700195312,21.104045867919922,-0.08877792209386826,1.124982833862304

In [29]:
t_zr.write('t_zr_0307.ecsv',overwrite=True)
t_zg.write('t_zg_0307.ecsv',overwrite=True)

In [28]:
t_zg[t_zg['Gaia_count']>1]

SourceID,filter,OID,RA,DEC,numobs,DELTA_SIG_MAG,CONST_CHI2,sf,rmse,LNP_SIG,period,period_c,power,power_c,mean,std,min,max,sigma_mean,sigma_std,gaussian1_err,gaussian2_err,gumbel_err,g2_flag,Pg1g2,Pgug2,Pg1gu,shape,upsilon_class,upsilon_prob,upsilon_flag,upsilon_period,identifier,dist,type,Gaia_count,multiflag,GaiaDist,fluxratio,Gmag,RFG,Plx,e_Plx,Plxflag,var,lc_var,var_flag
int32,str2,str16,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,str14,float64,float64,float64,str37,float64,str15,int32,int32,float64,float64,float64,float64,float64,float64,int32,str1,str1,bool
25,zg,806103100042687,5.0693889,59.3087403,237,92.99468231201172,7.836505926956435,1.512161804629138e-83,0.2091728299856186,1.3050029596712171e-13,365.20933973392897,1.0000143393856216,0.27259997355979004,0.4319465018424059,20.036285400390625,0.2091728299856186,18.759004592895508,20.42414093017578,-0.3137721121311188,2.870281219482422,0.005032831736797012,0.0007655728812316515,0.030898521657569875,1.0,0.013749444378141296,0.00037407861582017254,0.9856220212643277,g1,CEPH_Other,0.65,1.0,0.9947116227631579,--,nan,--,2,1,1.973932,1.1650830777292955,20.466309,132.148,1.6057,0.7252,1,V,V,False
25,zg,1809111300047902,5.0694621,59.3088523,36,40.49337387084961,5.178225498591696,8.048144074547564e-07,0.22259026765823364,0.00047935979507551194,0.17544479025567672,2800.0,0.5728030200438375,0.04952022327252487,20.041297912597656,0.22259026765823364,19.389726638793945,20.490066528320312,-0.3091849982738495,2.317760705947876,0.06488461442003488,0.021163611004506994,0.059744777164266294,1.0,0.060936121212942895,0.0714271704005417,0.45805178479963216,gu,DSCT,0.73,1.0,0.11622198129980171,--,nan,--,2,1,1.973932,1.1650830777292955,20.466309,132.148,1.6057,0.7252,1,V,V,False
55,zg,806103100036162,5.0724243,59.3110593,337,524.6041259765625,11.53433716011322,3.585358205401398e-226,0.09109293669462204,6.624513205261614e-05,0.9972441069114794,1.0000143393856216,0.09943739652561774,0.904815666787983,17.759519577026367,0.09109293669462204,17.35228729248047,18.282495498657227,-0.11940788477659225,3.39772629737854,0.021082675533131933,0.0009674376856350767,0.058600720395744474,1.0,0.0012778740354683071,0.0001663393556028403,0.8995923437308072,g1,RRL_ab,0.53,1.0,0.4986132790810157,[BGR2000] Br 1,0.179192,HII,2,1,0.977562,1.1336349112350974,20.799721,47.8212,nan,nan,0,V,V,False
55,zg,1809111300028565,5.07245,59.3110868,66,392.635986328125,14.967355206033764,8.396331195386733e-66,0.09816054254770279,0.001972449838350469,0.22232818693454928,2800.0,0.33767960153347515,0.09093510017475841,17.740047454833984,0.09816054254770279,17.36347198486328,17.94055938720703,-0.16121967136859894,3.8505306243896484,0.379122196310376,0.1487498025445291,0.17974030374835914,1.0,0.0868797174594248,0.34586527658886057,0.1729739946754486,gu,DSCT,0.5,1.0,0.05968903932818899,[BGR2000] Br 1,0.179192,HII,2,1,0.977562,1.1336349112350974,20.799721,47.8212,nan,nan,0,V,V,False
61,zg,806103100040276,5.067418,59.2966828,127,34.537837982177734,2.9826919894988544,0.0002645315332656795,0.2247643619775772,3.7800404304745114e-06,1.0321732994604866,1.0000110121384305,0.269655655474641,0.17275268370836522,20.80208396911621,0.2247643619775772,19.886661529541016,21.27700424194336,-0.16201622784137726,1.7484934329986572,0.06429252948295217,0.027821371972721838,0.09577891675859604,0.0,0.10470883703559108,0.04870495303435052,0.694010862155763,g1,CEPH_Other,0.55,1.0,1.0322700696219942,--,nan,--,2,1,0.380721,1.6046459566062432,19.862377,194.461,1.8337,0.6264,1,N,N,False
61,zg,1809111300042664,5.067473,59.2966488,15,13.316123962402344,1.3295706010789117,0.8108960576869506,0.18107299506664276,0.04263363261093376,0.10109222043491486,2800.0,0.7602083559250822,0.044869790029799385,20.748355865478516,0.18107299506664276,20.361892700195312,21.104045867919922,-0.08877792209386826,1.124982833862304